In [184]:
import pandas as pd
import numpy as np
from data_work import processed
from data_work import analyse_dataset
df=pd.read_csv("/Users/mac/Dev/data/dta_IoT/new2test.csv")
#delete all null columns
id=[16,17,21,22,23,24]
col=df.columns
for idx in id:
    df=df.drop(col[idx],axis=1)
data=processed(df,"subcategory ")
names=["attack","category","subcategory "]
analyse_dataset(data,names)

thống kê nhãn attack:
1 6095
0 931
thống kê nhãn category:
0 2994
1 931
3 1109
2 1992
thống kê nhãn subcategory :
6 1000
7 1000
3 931
2 995
4 992
5 1000
1 994
0 114


In [185]:
import torch.nn.functional as F
import torch
labels=torch.tensor(data["subcategory "])
labels_one_hot = F.one_hot(labels, 8)
labels_one_hot 


tensor([[0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 1],
        ...,
        [1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 1, 0]])

In [186]:
X=data.drop("subcategory ",axis=1)
X=torch.tensor(X.values)
data=torch.cat((X, labels_one_hot), dim=1)

In [187]:
import utils, torch, time, os, pickle
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

class generator(nn.Module):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : FC1024_BR-FC7x7x128_BR-(64)4dc2s_BR-(1)4dc2s_S
    def __init__(self, input_dim=100, output_dim=1, input_size=32, class_num=8):
        super(generator, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.input_size = input_size
        self.class_num = class_num

        self.fc = nn.Sequential(
            nn.Linear(self.input_dim + self.class_num, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024,512)
            nn.ReLU()
            nn.Linear(512 ,self.output_dim)
        )
        utils.initialize_weights(self)

    def forward(self, input, label):
        x = torch.cat([input, label], 1)
        x = self.fc(x)
        return x

class discriminator(nn.Module):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : (64)4c2s-(128)4c2s_BL-FC1024_BL-FC1_S
    def __init__(self, input_dim=1, output_dim=1, input_size=32, class_num=8):
        super(discriminator, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.input_size = input_size
        self.class_num = class_num

        self.fc = nn.Sequential(
            nn.Linear(self.input_dim+self.class_num, 512),
            nn.ReLU()
            nn.Linear(512,1024)
            nn.ReLU()
            nn.Linear(1024,512)
            nn.ReLU
            nn.Linear(512, self.output_dim),
            nn.Sigmoid(),
        )
        utils.initialize_weights(self)

    def forward(self, input, label):
        x = torch.cat([input, label], 1)
        x = self.fc(x)
        return x


class CGAN(object):
    def __init__(self, args):
        # parameters
        self.epoch = args.epoch
        self.batch_size = args.batch_size
        self.save_dir = args.save_dir
        self.result_dir = args.result_dir
        self.dataset = args.dataset
        self.log_dir = args.log_dir
        self.gpu_mode = args.gpu_mode
        self.model_name = args.gan_type
        self.input_size = args.input_size
        self.z_dim = 62
        self.class_num = args.n_class
        self.sample_num = self.class_num ** 2 
        # load dataset
        self.data_loader = DataLoader(self.dataset, batch_size=self.batch_size, shuffle=True)
        #data = self.data_loader.__iter__().__next__()[0]
        data=next(iter(self.data_loader))
        #option
        dim=int(data.shape[1])-8

        # networks init
        self.G = generator(input_dim=self.z_dim, output_dim=dim, input_size=self.input_size, class_num=self.class_num)
        self.D = discriminator(input_dim=dim, output_dim=1, input_size=self.input_size, class_num=self.class_num)
        self.G_optimizer = optim.Adam(self.G.parameters(), lr=args.lrG, betas=(args.beta1, args.beta2))
        self.D_optimizer = optim.Adam(self.D.parameters(), lr=args.lrD, betas=(args.beta1, args.beta2))

        self.BCE_loss = nn.BCELoss()

        print('---------- Networks architecture -------------')
        utils.print_network(self.G)
        utils.print_network(self.D)
        print('-----------------------------------------------')

        # fixed noise & condition
        self.sample_z_ = torch.zeros((self.sample_num, self.z_dim)) # z_dim tượng trưng cho noise vector, vector nhiều có kích thuóc dài bằng vector onehotencoded^2 của label 
        for i in range(self.class_num):
            self.sample_z_[i*self.class_num] = torch.rand(1, self.z_dim)#tạo một vector random có kích thuớc 1, z_dim ( chiều dài của vector nhiễu )
            for j in range(1, self.class_num):
                self.sample_z_[i*self.class_num + j] = self.sample_z_[i*self.class_num]

        temp = torch.zeros((self.class_num, 1))
        for i in range(self.class_num):
            temp[i, 0] = i

        temp_y = torch.zeros((self.sample_num, 1))
        for i in range(self.class_num):
            temp_y[i*self.class_num: (i+1)*self.class_num] = temp

        self.sample_y_ = torch.zeros((self.sample_num, self.class_num)).scatter_(1, temp_y.type(torch.LongTensor), 1)


    def train(self):
        self.train_hist = {}
        self.train_hist['D_loss'] = []
        self.train_hist['G_loss'] = []
        self.train_hist['per_epoch_time'] = []
        self.train_hist['total_time'] = []

        self.y_real_, self.y_fake_ = torch.ones(self.batch_size, 1), torch.zeros(self.batch_size, 1)

        self.D.train()
        print('training start!!')
        start_time = time.time()
        for epoch in range(self.epoch):
            self.G.train()
            epoch_start_time = time.time()
            for iter, da in enumerate(self.data_loader):
                x_=da[:,:28].float()
                y_=da[:,-8:].float()
                if iter == self.data_loader.dataset.__len__() // self.batch_size:
                    break

                z_ = torch.rand((self.batch_size, self.z_dim))
                #y_vec_ = torch.zeros((self.batch_size, self.class_num)).scatter_(1, y_.type(torch.LongTensor), 1)
                #y_fill_ = y_vec_.unsqueeze(2).unsqueeze(3).expand(self.batch_size, self.class_num, self.input_size, self.input_size)

                # update D network
                self.D_optimizer.zero_grad()

                D_real = self.D(x_, y_)
                D_real_loss = self.BCE_loss(D_real, self.y_real_)

                G_ = self.G(z_, y_)
                D_fake = self.D(G_, y_)
                D_fake_loss = self.BCE_loss(D_fake, self.y_fake_)

                D_loss = D_real_loss + D_fake_loss
                self.train_hist['D_loss'].append(D_loss.item())

                D_loss.backward()
                self.D_optimizer.step()

                # update G network
                self.G_optimizer.zero_grad()

                G_ = self.G(z_, y_)
                D_fake = self.D(G_, y_)
                G_loss = self.BCE_loss(D_fake, self.y_real_)
                self.train_hist['G_loss'].append(G_loss.item())

                G_loss.backward()
                self.G_optimizer.step()

                if ((iter + 1) % 100) == 0:
                    print("Epoch: [%2d] [%4d/%4d] D_loss: %.8f, G_loss: %.8f" %
                          ((epoch + 1), (iter + 1), self.data_loader.dataset.__len__() // self.batch_size, D_loss.item(), G_loss.item()))

            self.train_hist['per_epoch_time'].append(time.time() - epoch_start_time)
            with torch.no_grad():
                print("OK")
                

        self.train_hist['total_time'].append(time.time() - start_time)
        print("Avg one epoch time: %.2f, total %d epochs time: %.2f" % (np.mean(self.train_hist['per_epoch_time']),
              self.epoch, self.train_hist['total_time'][0]))
        print("Training finish!")


In [188]:
class Args:
    def __init__(self,data):
        self.epoch = 50
        self.batch_size = 32
        self.save_dir = '/Users/mac/Dev/source/article/FerderatedLearning/federated_learning/NonIID/model.h5'
        self.result_dir = '/Users/mac/Dev/source/article/FerderatedLearning/federated_learning/NonIID/result.txt'
        self.dataset = data
        self.log_dir = 'logs/'
        self.gpu_mode = True
        self.gan_type = 'infoGAN'
        self.input_size = 32
        self.z_dim = 62
        self.n_class = 8
        self.sample_num = self.n_class ** 2
        self.lrG=0.002
        self.lrD=0.002
        self.beta1=0.9
        self.beta2=0.99

In [189]:
arg=Args(data)
print(arg.dataset)

tensor([[5333, 2842,    7,  ...,    0,    1,    0],
        [5216, 2725,    7,  ...,    0,    1,    0],
        [6306, 3797,    0,  ...,    0,    0,    1],
        ...,
        [3998, 5464,    0,  ...,    0,    0,    0],
        [5798, 3307,    7,  ...,    0,    1,    0],
        [5379, 2888,    6,  ...,    0,    1,    0]])


In [190]:
model=CGAN(arg)

---------- Networks architecture -------------
generator(
  (fc): Sequential(
    (0): Linear(in_features=70, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=28, bias=True)
  )
)
Total number of parameters: 614940
discriminator(
  (fc): Sequential(
    (0): Linear(in_features=36, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Linear(in_features=1024, out_features=1, bias=True)
    (4): Sigmoid()
  )
)
Total number of parameters: 40961
-----------------------------------------------


In [191]:
model.train()

training start!!
Epoch: [ 1] [ 100/ 219] D_loss: 0.88499707, G_loss: 1.29847598
Epoch: [ 1] [ 200/ 219] D_loss: 0.62361205, G_loss: 1.78231609
OK
Epoch: [ 2] [ 100/ 219] D_loss: 0.43072447, G_loss: 2.00979185
Epoch: [ 2] [ 200/ 219] D_loss: 0.46461743, G_loss: 1.92128348
OK
Epoch: [ 3] [ 100/ 219] D_loss: 0.59615886, G_loss: 1.53919792
Epoch: [ 3] [ 200/ 219] D_loss: 0.67787862, G_loss: 1.46551633
OK
Epoch: [ 4] [ 100/ 219] D_loss: 0.54020572, G_loss: 2.39436412
Epoch: [ 4] [ 200/ 219] D_loss: 0.34877545, G_loss: 2.25819278
OK
Epoch: [ 5] [ 100/ 219] D_loss: 0.58744091, G_loss: 1.88523638
Epoch: [ 5] [ 200/ 219] D_loss: 0.90095377, G_loss: 1.07679880
OK
Epoch: [ 6] [ 100/ 219] D_loss: 0.77850556, G_loss: 1.76344192
Epoch: [ 6] [ 200/ 219] D_loss: 0.64233410, G_loss: 1.83246672
OK
Epoch: [ 7] [ 100/ 219] D_loss: 0.51161551, G_loss: 2.01502156
Epoch: [ 7] [ 200/ 219] D_loss: 0.83974361, G_loss: 1.74139845
OK
Epoch: [ 8] [ 100/ 219] D_loss: 0.62014532, G_loss: 1.64481616
Epoch: [ 8] [ 200

In [176]:
y=torch.tensor([1,2,3,4,5,6,7,0])
y=F.one_hot(y, 8)



In [177]:
y

tensor([[0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0]])

In [178]:
len=y.shape[0]
len

8

In [179]:
z= torch.rand(len,62)
yin=torch.tensor(yin)

/var/folders/0j/2h4f0bj12r775fg_s2v4hpqh0000gn/T/ipykernel_27098/2598516341.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yin=torch.tensor(yin)


In [180]:
mm=torch.cat([z,y],1)

In [181]:
data_fake=model.G(z,y)

In [182]:
data_fake

tensor([[-1.0996e+02, -1.2504e+02,  7.6761e-01,  2.5407e+00, -7.3504e-01,
         -1.0051e+02, -2.2982e+00,  5.2797e-01,  2.2641e+00, -1.1278e+00,
         -7.8942e+00, -1.0998e+02, -1.4136e+01, -5.1857e+01, -5.3497e+01,
         -1.5120e+01, -4.0590e+01, -8.3259e+00, -3.4016e+01, -1.4989e+00,
         -6.0830e+00,  8.5784e+00, -1.8214e+00,  6.8878e+01, -9.6606e+00,
         -4.3562e+00,  1.1937e+00, -3.4673e+00],
        [ 5.3926e+01,  8.4354e+01,  2.3195e-01,  1.1873e+00,  2.4098e+00,
         -4.3801e+01, -1.5839e-01, -5.9253e+00,  2.1230e-01, -4.4225e+00,
         -2.3346e+00,  7.0126e+01, -1.0444e+01, -2.0543e+01, -9.5574e+00,
          9.6565e-01, -6.0391e-01, -6.1603e+00, -6.8346e+00, -2.7305e-01,
          1.6098e+00,  1.2959e+00, -4.5213e-01,  2.3616e+01, -1.1262e+00,
          3.6198e-01, -3.3272e+00,  1.6809e+00],
        [ 8.6594e+01, -1.5694e+00, -3.1943e+00, -4.0194e-01,  1.7311e+00,
         -1.0148e+02, -1.3796e-01, -4.8547e+00,  2.9745e+00, -9.5748e+00,
         -2.84